In [99]:
import pandas as pd
import random


In [100]:

# Specify the file path
file_path = "all_seasons.csv"

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the head (first few rows) of the DataFrame
print("Head of the DataFrame:")
print(df.head())

# Display the tail (last few rows) of the DataFrame
print("\nTail of the DataFrame:")
print(df.tail())


Head of the DataFrame:
   Unnamed: 0        player_name team_abbreviation   age  player_height  \
0           0      Dennis Rodman               CHI  36.0         198.12   
1           1  Dwayne Schintzius               LAC  28.0         215.90   
2           2       Earl Cureton               TOR  39.0         205.74   
3           3        Ed O'Bannon               DAL  24.0         203.20   
4           4        Ed Pinckney               MIA  34.0         205.74   

   player_weight                      college country draft_year draft_round  \
0      99.790240  Southeastern Oklahoma State     USA       1986           2   
1     117.933920                      Florida     USA       1990           1   
2      95.254320                Detroit Mercy     USA       1979           3   
3     100.697424                         UCLA     USA       1995           1   
4     108.862080                    Villanova     USA       1985           1   

   ...  pts   reb  ast  net_rating  oreb_pct 

In [101]:
df['draft_year'] = df['season'].str.split('-').str[0].astype(int)

# Define the target year and the window size
target_year = 1996  # Replace with your desired target year
window_size = 5

# Calculate the start and end years of the window
start_year = target_year - window_size
end_year = target_year

# Filter the dataset to include only records within the 5-year window
filtered_df = df[(df['draft_year'] >= start_year) & (df['draft_year'] <= end_year)]

# Ensure the filtered dataset has at least 100 players
if len(filtered_df) < 100:
    print("There are not enough players within the specified window.")
else:
    # Randomly select 100 players from the filtered dataset
    random.seed(42)  # Set a random seed for reproducibility
    selected_players = random.sample(range(len(filtered_df)), 100)

    # Create a DataFrame containing the selected players
    selected_df = filtered_df.iloc[selected_players]

    # Display the selected players
    print(selected_df)


     Unnamed: 0       player_name team_abbreviation   age  player_height  \
327         327     Tom Gugliotta               MIN  27.0         208.28   
57           57  Hot Rod Williams               PHX  34.0         210.82   
12           12     Emanual Davis               HOU  28.0         195.58   
379         379      Jud Buechler               CHI  29.0         198.12   
140         140    Aaron Williams               VAN  25.0         205.74   
..          ...               ...               ...   ...            ...   
435         435      Martin Lewis               TOR  22.0         198.12   
358         358       Kenny Smith               DEN  32.0         190.50   
236         236    Sedale Threatt               HOU  35.0         187.96   
363         363     Jimmy Carruth               MIL  27.0         208.28   
138         138        A.C. Green               DAL  33.0         205.74   

     player_weight                          college country  draft_year  \
327     108.


## defining Player types    

In [ ]:
"""
1 point guard
    ast = 0.6
    ast_pct = 0.4

2 shooting guard
    pts = 0.8
    ast = 0.2

3 small forward
    net_rating = 0.5
    usg_pct = 0.5

4 forward
    pts = 0.4
    oreb_pct = 0.6

5 center
    reb = 0.7
    dreb_pct = 0.3 

"""

          player_name  playerRating
327     Tom Gugliotta       87.4974
57   Hot Rod Williams       65.6198
12      Emanual Davis       20.1309
379      Jud Buechler       59.4752
140    Aaron Williams       30.2042
..                ...           ...
435      Martin Lewis        7.6417
358       Kenny Smith       37.3330
236    Sedale Threatt       19.3770
363     Jimmy Carruth       -1.8982
138        A.C. Green       69.4483

[100 rows x 2 columns]


C:\Users\josh\AppData\Local\Temp\ipykernel_22144\2129467991.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['playerRating'] = (


In [104]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


# Define features (player characteristics) and target (player ratings)
features = selected_df[['gp', 'pts', 'reb', 'ast', 'net_rating', 'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct']].values
target = selected_df['playerRating'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a Sequential model
model = tf.keras.models.Sequential()

# Add an input layer with the number of features as input dimension
model.add(tf.keras.layers.Input(shape=(features.shape[1],)))

# Add one or more hidden layers with desired neurons and activation functions
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))

# Add an output layer with a single neuron (regression task)
model.add(tf.keras.layers.Dense(1))

# Compile the model with an appropriate optimizer, loss function, and evaluation metric
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=2)

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {mae}')

# Make predictions using the trained model
predictions = model.predict(X_test)

# You can use the predictions for further analysis or team optimization


Epoch 1/50
2/2 - 1s - loss: 1405.3169 - mae: 33.0343 - val_loss: 1712.5640 - val_mae: 36.4457 - 1s/epoch - 745ms/step
Epoch 2/50
2/2 - 0s - loss: 1232.3145 - mae: 30.8627 - val_loss: 1527.3644 - val_mae: 34.4300 - 97ms/epoch - 48ms/step
Epoch 3/50
2/2 - 0s - loss: 1066.6689 - mae: 28.6701 - val_loss: 1350.1509 - val_mae: 32.4436 - 88ms/epoch - 44ms/step
Epoch 4/50
2/2 - 0s - loss: 915.0015 - mae: 26.4358 - val_loss: 1183.2144 - val_mae: 30.4375 - 88ms/epoch - 44ms/step
Epoch 5/50
2/2 - 0s - loss: 779.9778 - mae: 24.2049 - val_loss: 1018.6310 - val_mae: 28.2866 - 88ms/epoch - 44ms/step
Epoch 6/50
2/2 - 0s - loss: 646.1665 - mae: 21.8778 - val_loss: 862.1849 - val_mae: 26.0418 - 80ms/epoch - 40ms/step
Epoch 7/50
2/2 - 0s - loss: 507.2743 - mae: 19.3284 - val_loss: 715.8251 - val_mae: 23.6904 - 90ms/epoch - 45ms/step
Epoch 8/50
2/2 - 0s - loss: 400.2001 - mae: 16.9086 - val_loss: 582.6825 - val_mae: 21.2581 - 83ms/epoch - 42ms/step
Epoch 9/50
2/2 - 0s - loss: 297.2334 - mae: 14.3751 - val